# Pytorch Implementation of EEGLearn - P. Bashivan

This notebook describes a short summary of Pytorch implementation of the models described in "Learning Representations from EEG with Deep Recurrent-Convolutional Neural Networks." Bashivan et al. at International conference on learning representations (2016).

The rest of the code is in the different python scripts of this repo.

All the codes have been inspired from the [original github](https://github.com/pbashivan/EEGLearn).

In [ ]:
import torch

# 检查CUDA是否可用
if torch.cuda.is_available():
    # 获取GPU设备数量
    device_count = torch.cuda.device_count()

    # 列出可用的GPU设备
    for i in range(device_count):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available. No GPU devices found.")


In [ ]:
# 检查CUDA1是否可用，如果可用则使用CUDA1
if torch.cuda.is_available():
    device = torch.device("cuda:3")
    print(f"Device: {device}")

## Librairies Import

In [ ]:
import numpy as np 
import scipy.io as sio
import torch
import os 

import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader,random_split

from Utils import *
from Models import *

torch.manual_seed(1234)
np.random.seed(1234)

import warnings
warnings.simplefilter("ignore")

## Loading the original Images 
The images have directly been taken from original implementation, given that they remain the same nevermind the implementation (Pytorch, Tensorflow, Theano).

In [ ]:
Mean_Images = sio.loadmat("Sample Data/images.mat")["img"] #corresponding to the images mean for all the seven windows
print(np.shape(Mean_Images)) 
Images = sio.loadmat("Sample Data/images_time.mat")["img"] #corresponding to the images mean for all the seven windows
print(np.shape(Images)) 
Label = (sio.loadmat("Sample Data/FeatureMat_timeWin")["features"][:,-1]-1).astype(int) #corresponding to the signal label (i.e. load levels). 0,1,2,3
print(np.shape(Label)) 
Patient_id = sio.loadmat("Sample Data/trials_subNums.mat")['subjectNum'][0] #corresponding to the patient id
print(np.shape(Patient_id))

In [ ]:
Mean_Images = sio.loadmat("Sample Data/images.mat")
print(Mean_Images)

In [ ]:
Images = sio.loadmat("Sample Data/images_time.mat")
print(Images)
all_zeros = np.all(Images["img"] == 0)
if all_zeros:
    print("All zeros")
else:
    print("Not all zeros")

In [ ]:
Label = (sio.loadmat("Sample Data/FeatureMat_timeWin")["features"])
print(Label[:,-1]-1)

In [ ]:
Patient_id = sio.loadmat("Sample Data/trials_subNums.mat")
print(Patient_id["subjectNum"])

## Loading patient dataset 
From the total data, we select the images corresponding patient. 

In [ ]:
print("Choose among the patient : "+str(np.unique(Patient_id)))

In [ ]:
choosen_patient = 9

## Introduction: BasicCNN
First Implementation of a CNN on the Mean Images from each patient

In [ ]:
train_part = 0.8
test_part = 0.2

batch_size = 32

In [ ]:
EEG = EEGImagesDataset(label=Label[Patient_id==choosen_patient], image=Mean_Images[Patient_id==choosen_patient])

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [ ]:
res = TrainTest_Model(BasicCNN, Trainloader, Testloader, n_epoch=50, learning_rate=0.001, print_epoch=-1, opti='Adam')

## Maxpool CNN
Build the Max-pooling model performing a maxpool over the 7 parallel convnets.

In [ ]:
train_part = 0.8
test_part = 0.2

batch_size = 32

In [ ]:
EEG = EEGImagesDataset(label=Label[Patient_id==choosen_patient], image=Images[Patient_id==choosen_patient])

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [ ]:
print('Begin Training for Patient '+str(choosen_patient))
res = TrainTest_Model(MaxCNN, Trainloader, Testloader, n_epoch=45, learning_rate=0.001, print_epoch=5, opti='Adam')

## Temp CNN
FBuild the Conv1D model performing a convolution1D over the 7 parallel convnets.

In [ ]:
print('Begin Training for Patient '+str(choosen_patient))
res = TrainTest_Model(TempCNN, Trainloader, Testloader, n_epoch=45, learning_rate=0.001, print_epoch=5, opti='Adam')

## LSTM CNN
Build the LSTM model applying a RNN over the 7 parallel convnets outputs

In [ ]:
EEG = EEGImagesDataset(label=Label[Patient_id==choosen_patient], image=Images[Patient_id==choosen_patient])

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [ ]:
print('Begin Training for Patient '+str(choosen_patient))
res = TrainTest_Model(LSTM, Trainloader, Testloader, n_epoch=45, learning_rate=0.0001, print_epoch=5, opti='Adam')

## Mix CNN
Build the LSTM model applying a RNN and a CNN over the 7 parallel convnets outputs

In [ ]:
EEG = EEGImagesDataset(label=Label[Patient_id==choosen_patient], image=Images[Patient_id==choosen_patient])

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [ ]:
print('Begin Training for Patient '+str(choosen_patient))
res = TrainTest_Model(Mix, Trainloader, Testloader, n_epoch=60, learning_rate=0.00001, print_epoch=5, opti='Adam')